# 3.1	Insert records from
42_District_wise_crimes_committed_against_women_2001_2012.csv into a table

-- Assuming your CSV file is comma-separated and has headers

CREATE TABLE IF NOT EXISTS crimes_against_women (
  State VARCHAR(255) NOT NULL,
  District VARCHAR(255) NOT NULL,
  Year INTEGER NOT NULL,
  Rape INTEGER,
  Kidnapping INTEGER,
  ... -- Add other columns from the CSV file if they exist
);

LOAD DATA LOCAL INFILE 'path/to/your/file.csv'
INTO TABLE crimes_against_women
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS; -- Skip the header row (assuming the first row has column names)

# 3.2	Write SQL query to find the highest number of rapes & Kidnappings that happened in which state, District, and year





SELECT 
  STATE/UT,
  District,
  Year,
  SUM(CASE WHEN CrimeType IN ('Rape''OTHER RAPE', 'KIDNAPPING & ABDUCTION','KIDNAPPING AND ABDUCTION OF WOMEN') THEN 1 ELSE 0 END) AS Rape_Kidnapping_Count
FROM dwc
GROUP BY STATE/UT, DISTRICT, YEAR
ORDER BY RAPE_Kidnapping_Count DESC
LIMIT 5

# 3.3	Write SQL query to find All the lowest number of rapes & Kidnappings that happened in which state, District, and year

SELECT 
  State,
  District,
  Year,
  SUM(CASE WHEN CrimeType IN ('RAPE''OTHER RAPE', 'KIDNAPPING & ABDUCTION','KIDNAPPING AND ABDUCTION OF WOMEN') THEN 1 ELSE 0 END) AS Rape_Kidnapping_Count
FROM YourTable
GROUP BY State, District, Year
HAVING Rape_Kidnapping_Count = (
  SELECT MIN(Rape_Kidnapping_Count)
  FROM YourTable
  GROUP BY State, District, Year
);

# 3.4	Insert records from 02_District_wise_crimes_committed_against_ST_2001_2012.csv into a new table



the SQL code to insert records from a CSV file named 02_District_wise_crimes_committed_against_ST_2001_2012.csv into a new table:

SQL
-- Assuming your CSV file is comma-separated and has headers

CREATE TABLE IF NOT EXISTS crimes_against_st (
  State VARCHAR(255) NOT NULL,
  District VARCHAR(255) NOT NULL,
  Year INTEGER NOT NULL,
  ... -- Add other columns from the CSV file if they exist
);

LOAD DATA LOCAL INFILE 'path/to/your/file.csv'
INTO TABLE crimes_against_st
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS; -- Skip the header row (assuming the first row has column names)
Use code with caution.


# 3.5	Write SQL query to find the highest number of dacoity/robbery in which district.



SELECT District,
       SUM(CASE WHEN CrimeType IN ('Dacoity', 'Robbery') THEN 1 ELSE 0 END) AS Dacoity_Robbery_Count
FROM YourTable
GROUP BY District
ORDER BY Dacoity_Robbery_Count DESC
LIMIT 1;

# 3.6	Write SQL query to find in which districts(All) the lowest number of murders happened

SELECT District
FROM 01_District_wise_crimes_committed_IPC_2001_2012
WHERE CrimeType = 'Murder'
GROUP BY District
HAVING COUNT(*) = (
  SELECT MIN(crime_count)
  FROM (
    SELECT District, COUNT(*) AS crime_count
    FROM 01_District_wise_crimes_committed_IPC_2001_2012
    WHERE CrimeType = 'Murder'
    GROUP BY District
  ) AS murder_counts
);

# 3.7	Write SQL query to find the number of murders in ascending order in district and yearwise.

SELECT District, Year, COUNT(*) AS Murder_Count
FROM 01_District_wise_crimes_committed_IPC_2001_2012
WHERE CrimeType = 'Murder'
GROUP BY District, Year
ORDER BY District, Year ASC;

# 3.8.1	Insert records of STATE/UT, DISTRICT, YEAR, MURDER, ATTEMPT TO MURDER, and RAPE columns only from 01_District_wise_crimes_committed_IPC_2001_2012.csv into a new table



-- Assuming your CSV file is comma-separated and has headers

CREATE TABLE IF NOT EXISTS crime_data (
  State_UT VARCHAR(255) NOT NULL,
  District VARCHAR(255) NOT NULL,
  Year INTEGER NOT NULL,
  Murder INTEGER,
  Attempt_Murder INTEGER,
  Rape INTEGER
);

LOAD DATA LOCAL INFILE 'path/to/your/file.csv'
INTO TABLE crime_data
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
IGNORE 1 ROWS ( -- Skip the header row
  SELECT State_UT, District, Year, Murder, Attempt_Murder, Rape
  FROM (
    SELECT * -- Select all columns from the CSV
    FROM YourCSVFile -- Replace with the actual CSV file alias
  ) AS tmp_data
  WHERE State_UT IS NOT NULL AND District IS NOT NULL AND  -- Ensure non-null values for these columns
        Year IS NOT NULL
);

# 3.8.2	Write SQL query to find which District in each state/ut has the highest number of murders yearwise. Your output should show STATE/UT, YEAR, DISTRICT, and MURDERS.

SELECT 
  State_UT,
  Year,
  District,
  Murder
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY State_UT, Year ORDER BY Murder DESC) AS rnk
  FROM 01_District_wise_crimes_committed_IPC_2001_2012
  WHERE CrimeType = 'Murder'
) AS ranked_data
WHERE rnk = 1;

# 3.8.3	Store the above data (the result of 3.2) in DataFrame and analyze districts that appear 3 or more than 3 years and print the corresponding state/ut, district, murders, and year in descending order.

import pandas as pd

# Sample data (replace with your actual data source)
data = {
    'State_UT': ['Andhra Pradesh', 'Andhra Pradesh', 'Andhra Pradesh', 'Arunachal Pradesh', 'Arunachal Pradesh', 'Assam', 'Assam', 'Assam'],
    'District': ['Guntur', 'Krishna', 'Guntur', 'East Kameng', 'West Kameng', 'Dhubri', 'Lakhimpur', 'Dhubri'],
    'Year': [2010, 2011, 2012, 2010, 2011, 2010, 2011, 2012],
    'Murder': [100, 120, 150, 50, 60, 80, 90, 100]
}

# Create pandas DataFrame
df = pd.DataFrame(data)

# Highest murder district per state/UT year-wise (similar to 3.8.2)
df['rank'] = df.groupby(['State_UT', 'Year'])['Murder'].transform(pd.Series.rank, ascending=False)
df_filtered = df[df['rank'] == 1]

# Districts appearing 3 or more years (using value counts)
three_or_more_years_districts = df_filtered['District'].value_counts()[df_filtered['District'].value_counts() >= 3]

# Filter data for those districts
result = df_filtered[df_filtered['District'].isin(three_or_more_years_districts.index.tolist())]

# Sort by Murder in descending order
result = result.sort_values(by='Murder', ascending=False)

# Print the DataFrame (without index)
print(result.to_string(index=False))

# 3.8.4	Use appropriate graphs to show your data (the result of 3.8.3)



import pandas as pd
import matplotlib.pyplot as plt

# Sample data (replace with your actual data source)
data = {
    'State_UT': ['Andhra Pradesh', 'Andhra Pradesh', 'Andhra Pradesh', 'Arunachal Pradesh', 'Arunachal Pradesh', 'Assam', 'Assam', 'Assam', 'Bihar', 'Bihar', 'Bihar'],
    'District': ['Guntur', 'Krishna', 'Guntur', 'East Kameng', 'West Kameng', 'Dhubri', 'Lakhimpur', 'Dhubri', 'Patna', 'Patna', 'Gaya'],
    'Year': [2010, 2011, 2012, 2010, 2011, 2010, 2011, 2012, 2010, 2011, 2012],
    'Murder': [100, 120, 150, 50, 60, 80, 90, 100, 110, 120, 130]
}

# Create pandas DataFrame
df = pd.DataFrame(data)

# Highest murder district per state/UT year-wise (similar to 3.8.2)
df['rank'] = df.groupby(['State_UT', 'Year'])['Murder'].transform(pd.Series.rank, ascending=False)
df_filtered = df[df['rank'] == 1]

# Districts appearing 3 or more years (using value counts)
three_or_more_years_districts = df_filtered['District'].value_counts()[df_filtered['District'].value_counts() >= 3]

# Filter data for those districts
result = df_filtered[df_filtered['District'].isin(three_or_more_years_districts.index.tolist())]

# Prepare data for charts
districts = result['District'].tolist()
murder_counts = result['Murder'].tolist()

# Bar chart
plt.figure(figsize=(10, 6))
plt.barh(districts, murder_counts, color='skyblue')
plt.xlabel('Number of Murders')
plt.ylabel('District')
plt.title('Murders in Districts Appearing 3+ Years (Descending)')
plt.gca().invert_yaxis()  # Invert y-axis to display most murders at the top
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Line chart (assuming year data is available)
if 'Year' in result.columns:
  years = result['Year'].unique()
  district_data = result.groupby('District')['Murder'].apply(list).reset_index()
  for district, murders in district_data.values.tolist():
    plt.plot(years, murders, marker='o', label=district)
  plt.xlabel('Year')
  plt.ylabel('Number of Murders')
  plt.title('Murder Trends in Districts Appearing 3+ Years')
  plt.legend()
  plt.grid(True)
  plt.tight_layout()
  plt.show()
